In [1]:
from collections import Counter #데이터의 개수를 정리할 수 있는 모듈
from konlpy.tag import Okt #형태소 분석기 호출

from soynlp.word import WordExtractor
from soynlp.tokenizer import LTokenizer
from soynlp.noun import LRNounExtractor

import pandas as pd #결과값을 데이터프레임 객체로 저장하기 위해 이용
# scikit-learn의 CountVectorizer를 사용하여 문서-단어 행렬을 생성
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [2]:
# Okt 객체 생성
okt = Okt()

In [4]:
# data 불러오기
data = pd.read_csv('youtube_channel_comments_cleaneddata.csv')
data.head()


,channelId,channelTitle,channelPublishedAt,subscriberCount,videoId,videoTitle,videoAuthorId,videoPublishedAt,duration,viewCount,likeCount,dislikeCount,commentId,commentAuthor,authorId,commentText,commentLikeCount,commentPublishedAt,parentCommentId,cleaned_commentText
0,UCMFk5S7g5DY-CZNVh_Kyz_A,약사가 들려주는 약 이야기,2017-03-02 02:28:38+00:00,1780000,qlLkiUU0o1A,우리 몸에 대장암이 생길때 나타나는 위험신호 대장암위험신호,UCMFk5S7g5DY-CZNVh_Kyz_A,2024-06-05 08:18:39+00:00,56,19765,339,0,UgxIwrowVdyFJFZloPx4AaABAg,@user-en3wv3ng8q,UCJZMfxEsrCsfkECnVQx1J5g,야채 많이드시고 가끔 요구르트 챙겨드시면\n어느정도 예방됨,0,2024-06-05 19:32:41+00:00,NaN,야채 많이 드시고 가끔 요구르트 챙겨드시면 어느 정도 예방됨
1,UCMFk5S7g5DY-CZNVh_Kyz_A,약사가 들려주는 약 이야기,2017-03-02 02:28:38+00:00,1780000,qlLkiUU0o1A,우리 몸에 대장암이 생길때 나타나는 위험신호 대장암위험신호,UCMFk5S7g5DY-CZNVh_Kyz_A,2024-06-05 08:18:39+00:00,56,19765,339,0,UgzzY7tk7b81qVjPlu14AaABAg,@user-fb4hd5lg1v,UCtTgUzxcR7KXdKUG4k3qTLA,제일 확실한건 변비 설사 반복 됩니다 꼭 검사해보셔요,2,2024-06-05 11:11:22+00:00,NaN,제일 확실한 건 변비 설사 반복 됩니다 꼭 검사해 보셔요
2,UCMFk5S7g5DY-CZNVh_Kyz_A,약사가 들려주는 약 이야기,2017-03-02 02:28:38+00:00,1780000,qlLkiUU0o1A,우리 몸에 대장암이 생길때 나타나는 위험신호 대장암위험신호,UCMFk5S7g5DY-CZNVh_Kyz_A,2024-06-05 08:18:39+00:00,56,19765,339,0,UgwWLF6ds6wSe2lU6DN4AaABAg,@inuyasha1017,UCEAlDUqmaikJpmie3nAIozg,좋은정보 감사합니다 😊😊😊,0,2024-06-05 10:43:46+00:00,NaN,좋은 정보 감사합니다
3,UCMFk5S7g5DY-CZNVh_Kyz_A,약사가 들려주는 약 이야기,2017-03-02 02:28:38+00:00,1780000,qlLkiUU0o1A,우리 몸에 대장암이 생길때 나타나는 위험신호 대장암위험신호,UCMFk5S7g5DY-CZNVh_Kyz_A,2024-06-05 08:18:39+00:00,56,19765,339,0,UgxDtXAN64oybfbaEDF4AaABAg,@user-cj6rk5qn9o,UC2HHPaUByk-R9ykbP3yjsaQ,걸리면 하느님 만나로 올라갈게요,1,2024-06-05 10:11:54+00:00,NaN,걸리면 하느님 만나로 올라갈게요
4,UCMFk5S7g5DY-CZNVh_Kyz_A,약사가 들려주는 약 이야기,2017-03-02 02:28:38+00:00,1780000,qlLkiUU0o1A,우리 몸에 대장암이 생길때 나타나는 위험신호 대장암위험신호,UCMFk5S7g5DY-CZNVh_Kyz_A,2024-06-05 08:18:39+00:00,56,19765,339,0,Ugx946j7jz6kQr-vDsh4AaABAg,@user-ye5ne4mc6d,UCQ0eOpBzyy1vgyF4nUTAb_Q,이중에 하나도 해당 안됐는데\n대장암 4기..,10,2024-06-05 08:33:27+00:00,NaN,이 중에 하나도 해당 안 됐는데 대장암 4기


In [5]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56405 entries, 0 to 56404
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   channelId            56405 non-null  object
 1   channelTitle         56405 non-null  object
 2   channelPublishedAt   56405 non-null  object
 3   subscriberCount      56405 non-null  int64 
 4   videoId              56405 non-null  object
 5   videoTitle           56405 non-null  object
 6   videoAuthorId        56405 non-null  object
 7   videoPublishedAt     56405 non-null  object
 8   duration             56405 non-null  int64 
 9   viewCount            56405 non-null  int64 
 10  likeCount            56405 non-null  int64 
 11  dislikeCount         56405 non-null  int64 
 12  commentId            56405 non-null  object
 13  commentAuthor        56277 non-null  object
 14  authorId             56405 non-null  object
 15  commentText          56405 non-null  object
 16  comm

In [6]:
# data를 채널아이디와 정제된 댓글만 남김
data = data[['channelId', 'videoPublishedAt','cleaned_commentText']]
data.head()
# CSV 파일로 저장
# data.to_csv('cleaned_comments.csv', index=False, encoding='utf-8-sig')

,channelId,videoPublishedAt,cleaned_commentText
0,UCMFk5S7g5DY-CZNVh_Kyz_A,2024-06-05 08:18:39+00:00,야채 많이 드시고 가끔 요구르트 챙겨드시면 어느 정도 예방됨
1,UCMFk5S7g5DY-CZNVh_Kyz_A,2024-06-05 08:18:39+00:00,제일 확실한 건 변비 설사 반복 됩니다 꼭 검사해 보셔요
2,UCMFk5S7g5DY-CZNVh_Kyz_A,2024-06-05 08:18:39+00:00,좋은 정보 감사합니다
3,UCMFk5S7g5DY-CZNVh_Kyz_A,2024-06-05 08:18:39+00:00,걸리면 하느님 만나로 올라갈게요
4,UCMFk5S7g5DY-CZNVh_Kyz_A,2024-06-05 08:18:39+00:00,이 중에 하나도 해당 안 됐는데 대장암 4기


In [7]:
# data 크기확인
data.shape

(56405, 3)

In [8]:
# konlpy의 Okt 형태소 분석기 이용 토크나이저 구축
def okt_tokenize(text):
    # 텍스트를 형태소 분석하여 품사 태깅
    tagged = okt.pos(text)
    # 명사, 형용사, 부사, 동사만 추출하고, 한 글자 품사는 제외
    tokens = [word for word, tag in tagged if tag in ['Noun', 'Adjective', 'Adverb'] and len(word) > 1]
    return tokens

In [9]:
# soynlp를 적용하기 위해 데이터 훈련
# NaN 값 제거
data = data.dropna(subset=['cleaned_commentText'])

# 또는

# NaN 값 공백 문자열로 대체
data['cleaned_commentText'] = data['cleaned_commentText'].fillna('')

# soynlp 훈련
word_extractor = WordExtractor(min_frequency=20,
    min_cohesion_forward=0.05, 
    min_right_branching_entropy=0.0)
word_extractor.train(data['cleaned_commentText'])
word = word_extractor.extract()

# 코헤전스 점수 계산
cohesion_score = {word:score.cohesion_forward for word, score in word.items()}



training was done. used memory 0.558 Gbory 0.388 Gb
all cohesion probabilities was computed. # words = 12685
all branching entropies was computed # words = 55316
all accessor variety was computed # words = 55316


In [10]:
# soynlp의 WordExtractor로부터 얻은 word_score_table을 이용하여 
# LTokenizer 또는 MaxScoreTokenizer를 구성
# 명사를 추출하기 위해 LRNounExtractor_v2 사용
noun_extractor = LRNounExtractor()
nouns = noun_extractor.train_extract(data['cleaned_commentText']) # list of str like

noun_scores = {noun:score.score for noun, score in nouns.items()}
combined_scores = {noun:score + cohesion_score.get(noun, 0)
    for noun, score in noun_scores.items()}
combined_scores.update(
    {subword:cohesion for subword, cohesion in cohesion_score.items()
    if not (subword in combined_scores)}
)

# 특정 복합명사에 대해 높은 점수 부여
custom_scores = {
    '요구르트': 1.0,  # 예시 요구르트
    '브로멜라인': 1.0,
    '멜라토닌': 1.0,
    '베르베린': 1.0,
    '커큐민' : 1.0,
    '아이스크림' : 1.0
}

# 점수 테이블 업데이트
combined_scores.update(custom_scores)
tokenizer = LTokenizer(scores=combined_scores)

[Noun Extractor] used default noun predictor; Sejong corpus predictor
[Noun Extractor] used noun_predictor_sejong
[Noun Extractor] All 2398 r features was loaded
[Noun Extractor] scanning was done (L,R) has (27716, 14253) tokens
[Noun Extractor] building L-R graph was done
[Noun Extractor] 4577 nouns are extracted


In [11]:
# 스톱워드 목록 정의
stopwords = {'많이', '하지만', '그러나', '저는', '제가', '보셔요', '중에', '됐는데','먹는', '먹어', '먹어도', 
             '먹으면', 'bandusband83355090', '하시면', '보고', '그리고', '되는', '됩니', '드시', '11', 'yakstory',
             '으로', '하고', '좋은', '있습니다', 'https', 'http', '입니다', '하는', '해서', '정도', '네이버', '상담은', '약사'
             'NM', 'nm', '에서', '하세요', '있어요', '있는', '같이', '입니다', '먹고', '먹어',
             'ㅎㅎ', '좋은', '같이', '있는데', '저도', '해요',  '바르고', '있는데','ezyaksa', '으로',
             '있어요', '입니다', '있는', '하는', '하세요', '해주셔서',
             '괜찮', '보다', '하면', '들어', '하나', 'NM', '항상', '해도', '하게',
             '없이', '이네요', '같습니다', '해도', '될까요', '안녕', 'be', 'youtu', 'ya', 
             'ngsnack101', '괜찮', '혹시', '부탁드립니다', '하시는', '어떤', '하셔도',
             '그런데', '이런', '저런', '합니다', '많은', '많이', '정말', '너무'}
def tokenize_with_stopwords(text):
    # 텍스트를 토크나이징
    tokens = tokenizer.tokenize(text)
    # 스톱워드 제거 및 한 글자 토큰 제거
    filtered_tokens = [token for token in tokens if token not in stopwords and len(token) > 1]
    return filtered_tokens

In [12]:
# 텍스트 토크나이저 적용

# data['tokenized'] = data['cleaned_commentText'].apply(okt_tokenize)
# soynlp의 LTokenizer를 이용한 토크나이저 적용
data['tokenized'] = data['cleaned_commentText'].apply(tokenize_with_stopwords)
data.head()



,channelId,videoPublishedAt,cleaned_commentText,tokenized
0,UCMFk5S7g5DY-CZNVh_Kyz_A,2024-06-05 08:18:39+00:00,야채 많이 드시고 가끔 요구르트 챙겨드시면 어느 정도 예방됨,"[야채, 가끔, 요구르트, 챙겨, 드시면, 어느, 예방]"
1,UCMFk5S7g5DY-CZNVh_Kyz_A,2024-06-05 08:18:39+00:00,제일 확실한 건 변비 설사 반복 됩니다 꼭 검사해 보셔요,"[제일, 확실, 변비, 설사, 반복, 검사]"
2,UCMFk5S7g5DY-CZNVh_Kyz_A,2024-06-05 08:18:39+00:00,좋은 정보 감사합니다,"[정보, 감사]"
3,UCMFk5S7g5DY-CZNVh_Kyz_A,2024-06-05 08:18:39+00:00,걸리면 하느님 만나로 올라갈게요,"[걸리, 하느님, 만나로, 올라, 갈게요]"
4,UCMFk5S7g5DY-CZNVh_Kyz_A,2024-06-05 08:18:39+00:00,이 중에 하나도 해당 안 됐는데 대장암 4기,"[해당, 대장암, 4기]"


In [13]:
# 채널별로 데이터 그룹화 및 토큰 리스트 합치기
grouped_data = data.groupby('channelId')['tokenized'].agg(lambda x: [token for sublist in x for token in sublist])

# 각 채널의 토큰 리스트를 사용하여 문서-단어 행렬을 생
# CountVectorizer 초기화 (토큰 리스트를 직접 처리하도록 설정)
vectorizer = CountVectorizer(tokenizer=lambda x: x, lowercase=False, preprocessor=lambda x: x)


In [14]:
grouped_data

channelId
UC3iSLVH0MxHfwO69oHKpvog    [리틀, 약사님, 최고, 이십니다, 감사, 비문증, 브로, 멜라, 파파인에, 대해서...
UC6ggXTuBVchhwHeQ12Ita1w    [Ph, ytosome, 베르, 베린, 복용, 방법, 안내, 복용, 이미, 아시겠지...
UCCMFTDGarjgZLc1DlIbbvRg    [갑니다, 감사, se, up, the, su, btitles, to, autotr...
UCMFk5S7g5DY-CZNVh_Kyz_A    [야채, 가끔, 요구르트, 챙겨, 드시면, 어느, 예방, 제일, 확실, 변비, 설사...
UCY-mXLM6DsS9cmSwlh0tqSA    [담백, 하루, 아르기닌, 변비, 유산균, 공구, 오픈, 구매, 링크, 트랜짓, 유...
Name: tokenized, dtype: object

In [15]:
from collections import Counter

# 각 채널별로 상위 20개 단어 추출
top_words_per_channel = {}
for channel_id, tokens in grouped_data.items():
    # 토큰의 빈도수 계산
    token_counts = Counter(tokens)
    # 가장 빈번한 30개 단어 추출
    top_words = token_counts.most_common(30)
    top_words_per_channel[channel_id] = top_words

In [16]:
# 결과 출력
for channel_id, words in top_words_per_channel.items():
    print(f"Channel {channel_id}:")
    for word, count in words:
        print(f"{word}: {count}")
    print("\n")

Channel UC3iSLVH0MxHfwO69oHKpvog:
감사: 1317
비타: 626
마그네슘: 613
제품: 599
영양제: 573
리틀: 552
복용: 538
영상: 521
밴드: 516
약사님: 491
오메가: 484
약사: 451
섭취: 403
건강: 401
효과: 358
도움: 348
하루: 336
정보: 333
답변: 309
유튜: 302
생각: 291
자세: 291
음식: 283
사람: 274
문제: 270
이용: 249
불편: 246
다른: 241
커피: 239
대한: 235


Channel UC6ggXTuBVchhwHeQ12Ita1w:
감사: 706
복용: 678
약사님: 557
영상: 329
제품: 273
섭취: 268
영양제: 244
효과: 236
정보: 233
건강: 178
선생님: 176
베르: 175
멜라토: 171
피세틴: 166
도움: 164
추천: 143
미국: 133
베린: 124
비타: 123
구매: 119
하루: 118
아침: 116
레스베라트롤: 108
피부: 107
궁금: 106
라이: 104
구입: 94
함께: 87
어떤: 85
Pharm: 83


Channel UCCMFTDGarjgZLc1DlIbbvRg:
감사: 88
약사님: 42
크림: 37
효과: 37
이지약사님: 31
제품: 30
멜라토: 25
피부: 22
정보: 21
멜라: 21
영상: 20
발라: 19
이지: 15
바르: 15
부위: 15
약국: 14
처방: 14
색소: 14
궁금: 14
쓰고: 14
오랜만: 13
발라도: 13
흡수: 12
여드름: 12
자외선: 12
흉터: 12
톡톡: 11
노사: 11
함량: 11
해주: 11


Channel UCMFk5S7g5DY-CZNVh_Kyz_A:
감사: 8550
섭취: 5693
영양제: 3680
오메가: 3674
비타: 3640
제품: 3448
약사님: 3259
복용: 3193
영상: 2882
도움: 2480
시청: 2374
추천: 2267
생각: 1848
마그네슘: 1831
말씀: 1661
부탁드립니

In [ ]:
vectorizer

In [ ]:
# channelId별로 LDA를 위해 5개의 문서-단어 행렬 생성

document_term_matrix = vectorizer.fit_transform(grouped_data)



In [ ]:
# LDA 모델 초기화 및 학습
lda_model = LatentDirichletAllocation(n_components=5, random_state=0)  # n_components는 토픽 수
lda_model.fit(document_term_matrix)

In [ ]:
# 학습된 LDA 모델로부터 각 토픽의 주요 단어들을 확인하고, 각 채널의 주요 토픽을 출력합니다.
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)

tf_feature_names = vectorizer.get_feature_names_out()
print_top_words(lda_model, tf_feature_names, 10)

# 각 채널의 주요 토픽 출력
channel_topics = lda_model.transform(document_term_matrix)
for channel_idx, topic_dist in enumerate(channel_topics):
    top_topic = topic_dist.argmax()
    print(f"Channel {grouped_data.index[channel_idx]} 주요 토픽: {top_topic}")